# 🎬 RotoTrackID — Colab で動画からアルファPNGを生成

このNotebookは次のフローを実現します：

1. 動画をアップロード
2. YOLO + ByteTrack でオブジェクトを追跡
3. ID付きプレビュー動画を生成
4. 特定IDを選択
5. 指定IDのアルファPNGを書き出し
6. ZIPでダウンロード

👉 すべてブラウザで実行できます（GPU推奨）


## 🧹 セッション初期化（必ず最初に実行）

Colabはセッションを引き継ぐため、前回の解析データが残っていると
誤動作する場合があります。以下を実行して環境をクリーンにします。


In [ ]:
# ===== RotoTrackID Colab Reset =====

import os
import shutil
import gc
import torch

print("Resetting Colab environment...")

# 1. GPUメモリ開放
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

# 2. 作業ディレクトリをクリーン
for d in ["result", "alpha", "__pycache__", ".ultralytics"]:
    if os.path.exists(d):
        shutil.rmtree(d, ignore_errors=True)

# 3. Pythonの参照を破棄
gc.collect()

# 4. 古いYOLO/SAMモデルのキャッシュを消す（バグ防止）
cache_dirs = [
    os.path.expanduser("~/.cache/ultralytics"),
    os.path.expanduser("~/.cache/torch"),
]

for d in cache_dirs:
    if os.path.exists(d):
        shutil.rmtree(d, ignore_errors=True)

print("Colab reset done.")


## 📥 ライブラリインストール

必要なライブラリをインストールします。
Colab の場合は毎回実行してください。


In [ ]:
!git clone https://github.com/nameshigawa/RotoTrackID
%cd RotoTrackID
!pip install -r requirements.txt

## 🤖 モデルダウンロード

YOLO11m と SAM モデルを準備します。
初回はダウンロードに数分かかります（数百MB）。


In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11m.pt")

!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/sam_l.pt

## 📤 入力動画をアップロード

解析したい動画ファイル（mp4 / mov など）をアップロードしてください。


In [ ]:
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

## 🔍 動画をトラッキング解析

YOLO + ByteTrack を使ってオブジェクトを追跡します。
IDと label の一覧も返ります。


In [ ]:
from analyze import analyze_video

analyze_video(video_path, write_annotated=True)

base_dir = os.path.dirname(video_path) or "."
base_name = os.path.splitext(os.path.basename(video_path))[0]
output_dir = os.path.join(base_dir, f"{base_name}_outputs")
out_video = os.path.join(output_dir, f"{base_name}_annot_analysis.mp4")

## 📥 プレビュー動画をダウンロード

追跡結果を反映した動画ファイルをローカルにダウンロードします。


In [ ]:
from google.colab import files

files.download(out_video)

## 🆔 対象IDを選択

`tracks` には検出されたオブジェクトごとの ID とラベルが入っています。
例: `[{'id': 3, 'label': 'person'}, ...]`

ここに **切り抜きたいIDを入力**してください。


In [ ]:
selected_id = 3  # ← プレビュー動画を見て数値を書き換えてください
print("Selected ID =", selected_id)

## ✂️ SAM で指定IDのアルファPNGを生成

選んだIDのアルファ付き PNG 連番を書き出します。


In [ ]:
import cv2
import time
from tqdm.notebook import tqdm
from alpha_export import export_alpha_by_id

# --- ① 動画の総フレーム数を取得 ---
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

# --- ② Colab用 Progress UI ---
class ColabProgress:
    def __init__(self, total):
        self.start = time.time()
        self.pbar = tqdm(total=total, desc="SAM Alpha Export", unit="frame")

    def __call__(self, current, total, elapsed=None):
        done = current + 1
        self.pbar.n = done

        now = time.time()
        elapsed = now - self.start
        fps = done / elapsed if elapsed > 0 else 0
        remaining = (total - done) / fps if fps > 0 else 0

        self.pbar.set_postfix({
            "elapsed": f"{elapsed:.1f}s",
            "eta": f"{remaining:.1f}s",
            "fps": f"{fps:.2f}"
        })
        self.pbar.refresh()

    def close(self):
        self.pbar.close()

progress = ColabProgress(total_frames)

out_dir = export_alpha_by_id(
    video_path=video_path,
    target_id=selected_id,
    out_dir="alpha",
    sam_model="sam_l.pt",
    progress_cb=progress
)

progress.close()


## 📦 生成された PNG を ZIP でダウンロード


In [ ]:
!zip -r roto_alpha_{selected_id}.zip alpha
files.download(f"roto_alpha_{selected_id}.zip")